In [52]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [53]:
column_names = list(range(0,16))
column_names.append('Classification')

df = pd.read_csv("zoo.data", names = column_names) 
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Classification
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [54]:
df.dtypes

0                 int64
1                 int64
2                 int64
3                 int64
4                 int64
5                 int64
6                 int64
7                 int64
8                 int64
9                 int64
10                int64
11                int64
12                int64
13                int64
14                int64
15                int64
Classification    int64
dtype: object

In [55]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

0                 0
1                 0
2                 0
3                 0
4                 0
5                 0
6                 0
7                 0
8                 0
9                 0
10                0
11                0
12                0
13                0
14                0
15                0
Classification    0
dtype: int64

In [56]:
df.nunique()

0                 2
1                 2
2                 2
3                 2
4                 2
5                 2
6                 2
7                 2
8                 2
9                 2
10                2
11                2
12                6
13                2
14                2
15                2
Classification    7
dtype: int64

In [57]:
encode_columns = [0,1,2,3,4,5,6,7,8,9,10,11,13,14,'Classification']
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
for columns in encode_columns:
    df[columns] = label_encoder.fit_transform(df[columns])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Classification
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,0
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,0
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,3
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,0
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,0
97,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,5
98,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,0
99,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,6


In [58]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [59]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (16,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(7, activation='softmax'))

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0


for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)

    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train)
    test_y = tf.keras.utils.to_categorical(y_test)

            
    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
3/3 [==============================] - 4s 767ms/step - loss: 1.6926 - accuracy: 0.5250 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.2601 - val_accuracy: 0.7619 - val_precision: 1.0000 - val_recall: 0.0476
Epoch 2/50
3/3 [==============================] - 0s 32ms/step - loss: 1.4242 - accuracy: 0.6750 - precision: 0.9286 - recall: 0.1625 - val_loss: 1.0413 - val_accuracy: 0.7619 - val_precision: 0.8182 - val_recall: 0.4286
Epoch 3/50
3/3 [==============================] - 0s 32ms/step - loss: 1.2402 - accuracy: 0.7000 - precision: 0.8621 - recall: 0.3125 - val_loss: 0.9003 - val_accuracy: 0.7619 - val_precision: 0.8462 - val_recall: 0.5238
Epoch 4/50
3/3 [==============================] - 0s 36ms/step - loss: 1.0691 - accuracy: 0.7250 - precision: 0.9310 - recall: 0.3375 - val_loss: 0.7829 - val_accuracy: 0.7619 - val_precision: 1.0000 - val_recall: 0.5238
Epoch 5/50
3/3 [==============================] - 0s 41ms/step - loss: 0.9225 - accuracy: 0.7875 - precisio

Epoch 38/50
3/3 [==============================] - 0s 30ms/step - loss: 0.0228 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.1106 - val_accuracy: 0.9524 - val_precision: 0.9524 - val_recall: 0.9524
Epoch 39/50
3/3 [==============================] - 0s 37ms/step - loss: 0.0207 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.1152 - val_accuracy: 0.9524 - val_precision: 0.9524 - val_recall: 0.9524
Epoch 40/50
3/3 [==============================] - 0s 30ms/step - loss: 0.0191 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.1203 - val_accuracy: 0.9524 - val_precision: 0.9524 - val_recall: 0.9524
Epoch 41/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0179 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.1244 - val_accuracy: 0.9524 - val_precision: 0.9524 - val_recall: 0.9524
Epoch 42/50
3/3 [==============================] - 0s 28ms/step - loss: 0.0169 - accuracy: 1.0000 - precision: 1

3/3 [==============================] - 0s 48ms/step - loss: 0.0047 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 18/50
3/3 [==============================] - 0s 38ms/step - loss: 0.0046 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 19/50
3/3 [==============================] - 0s 34ms/step - loss: 0.0044 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 20/50
3/3 [==============================] - 0s 36ms/step - loss: 0.0043 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0027 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 21/50
3/3 [==============================] - 0s 30ms/step - loss: 0.0041 - accuracy: 1.0000 - precision: 1.0000 - reca

Epoch 4/50
3/3 [==============================] - 0s 50ms/step - loss: 0.0016 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0019 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/50
3/3 [==============================] - 0s 70ms/step - loss: 0.0015 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0019 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 6/50
3/3 [==============================] - 0s 116ms/step - loss: 0.0015 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 7/50
3/3 [==============================] - 0s 106ms/step - loss: 0.0014 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 8/50
3/3 [==============================] - 0s 33ms/step - loss: 0.0014 - accuracy: 1.0000 - precision: 1.00

Epoch 41/50
3/3 [==============================] - 0s 37ms/step - loss: 7.5193e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 42/50
3/3 [==============================] - 0s 34ms/step - loss: 7.4062e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 43/50
3/3 [==============================] - 0s 34ms/step - loss: 7.2984e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 44/50
3/3 [==============================] - 0s 40ms/step - loss: 7.2464e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 45/50
3/3 [==============================] - 0s 40ms/step - loss: 7.0299e-04 - accuracy: 1

In [60]:
print('Average Accuracy Score: ', sum_acc_score/10)
print('Average Precision Score: ', sum_prec_score/10)
print('Average Recall Score: ', sum_recall_score/10)
print('Average F1-Score: ', sum_f1_score/10)



Average Accuracy Score:  0.49523809523809526
Average Precision Score:  0.49523809523809526
Average Recall Score:  0.49523809523809526
Average F1-Score:  0.49523809523809526
